In [4]:
import requests
import pandas as pd
import csv

# 1. Obtener lista de países
resp = requests.get( "https://api.worldbank.org/v2/country", params={"format": "json", "per_page": 400} )
data = resp.json()

#Data[0] estan los metadatos utilizamos el data[1]
paises = data[1]

df_countries = pd.DataFrame([
    {"name": p["name"], "iso3Code": p["id"], "region": p["region"]["value"]}
    for p in paises
])
# Quitamos por regiones 
df_countries = df_countries[df_countries["region"] != "Aggregates"].reset_index(drop=True)

#----IMPORTANTE!!!!------Comenta y desconecta segun convenga.

descargar = True 
# descargar = False

if not descargar:
    print( "Comprueba si esta el CSV desgargado, si no lo está, comenta y descomenta descargar  ")

else:
# 3. Creamos el CSV y escribimos los datos
    with open("data/datos.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerow(["country", "iso3", "year", "value"])  # Nomnre de las columnas
    
        # 4. Recorrer países
        for idx, row in df_countries.iterrows():
            iso3 = row["iso3Code"]
            name = row["name"]
    
            url = f"https://api.worldbank.org/v2/country/{iso3}/indicator/{"NY.GDP.PCAP.CD"}"
            params = {"format": "json","per_page": 20000,"date": "1980:2023"}
    
            resp = requests.get(url, params={"format": "json","per_page": 20000,"date": "1980:2023"})
            data = resp.json()
    
            # Recorrer años disponibles
            for entry in data[1]:
                value = entry["value"]
                year = entry["date"]
    
                writer.writerow([name, iso3, year, value])
            print(f" Datos guardados {name}")

 Datos guardados Aruba
 Datos guardados Afghanistan
 Datos guardados Angola
 Datos guardados Albania
 Datos guardados Andorra
 Datos guardados United Arab Emirates
 Datos guardados Argentina
 Datos guardados Armenia
 Datos guardados American Samoa
 Datos guardados Antigua and Barbuda
 Datos guardados Australia
 Datos guardados Austria
 Datos guardados Azerbaijan
 Datos guardados Burundi
 Datos guardados Belgium
 Datos guardados Benin
 Datos guardados Burkina Faso
 Datos guardados Bangladesh
 Datos guardados Bulgaria
 Datos guardados Bahrain
 Datos guardados Bahamas, The
 Datos guardados Bosnia and Herzegovina
 Datos guardados Belarus
 Datos guardados Belize
 Datos guardados Bermuda
 Datos guardados Bolivia
 Datos guardados Brazil
 Datos guardados Barbados
 Datos guardados Brunei Darussalam
 Datos guardados Bhutan
 Datos guardados Botswana
 Datos guardados Central African Republic
 Datos guardados Canada
 Datos guardados Switzerland
 Datos guardados Channel Islands
 Datos guardados Chil